In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

!cp /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/train.csv /content
!cp /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/test.csv /content

In [ ]:
import pandas as pd

prefix = '/content/'
test_df = pd.read_csv(prefix + 'test.csv', header=None) # re-name test.csv 
test_df=test_df.drop(index=0)

test_df = pd.DataFrame({
    'id':test_df[0],
    'text': test_df[1],
    'labels': ''
})

# display 
test_df.head()

In [ ]:
!pip install allennlp allennlp-models

!pip install nltk
!pip install tqdm
!pip install spacy 
!python -m spacy download en_core_web_sm

In [ ]:
from allennlp.predictors.predictor import Predictor
import allennlp_models.structured_prediction
predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/elmo-constituency-parser-2020.02.10.tar.gz")

In [ ]:
prediction = predictor.predict(
        sentence = "'Age may have dimmed their employment prospects, but older people often had the financial firepower to start up their own businesses - though should they have chosen to embark upon a second career, it would have been good doing something that they always wanted to do.'"
)
#prediction

START AGAIN

In [ ]:
import string
def extract_modal(obj):
    tags = list() 
    parts = list()
    words = list()
    nouns = list()
    verbs = list()

    includes= list()
    cutouts = list()

    conseqs = list()
    anteced = list()

    def extract(obj):

      if isinstance(obj, dict):
        for k, v in obj.items():
          if k=='nodeType':
            tags.append(v)
          if k=='word':
            words.append(v)
          #if k=='children': # as below
          if isinstance(v, (dict, list)):
            extract(v)
        
      elif isinstance(obj, list):
        for item in obj:
          if isinstance(item, dict):
            extract(item)

            if len(words)>0:
              word = item['word'].rstrip()
              zoom = 0
              index=-1
              if 'SBAR' in tags: 
                index = max(idx for idx, val in enumerate(tags) if val=='SBAR')
                zoom = index
              if 'S' in tags:
                index = max(idx for idx, val in enumerate(tags) if val=='S')
                if index > zoom:
                  zoom = index
              head = words[zoom]
              #print(tags, zoom, head)
              if (
                  'would'==word.lower() or 'wouldn'==word.lower() or 'wouldn\'t'==word.lower() or 'woulnd\'t'==word.lower() or 
                  'could'==word.lower() or 'could\'t'==word.lower() or 'coulnd\'t'==word.lower() or #'can'==word.lower() or
                  'might'==word.lower() or 'mightn\'t'==word.lower() or 'mighn\'t'==word.lower()  or 'may'==word.lower() or
                  #'should'==word.lower() or 'shouldn\'t'==word.lower() or 'shoulnd\'t'==word.lower() or 
                  'd'==word.lower() or word.lower().rstrip().endswith('\'d') or 'ought'==word.lower() or
                  'will have been' in word.lower() or 'would\'ve' in word.lower() or 'wouldn\'ve' in word.lower()
              ):
                phrase = scrap_from(obj,item)
                #print(phrase,'|',head)
                if phrase not in head or len(word_tokenize(head)) < 2:
                  head = phrase 
                parts.append({'tag':'MD','word':head,'text':obj})
                includes.append(word)
      
              #if 'MD' in tags:    
              #  if word not in string.punctuation:  
              #    parts.append({'tag':'MD','word':word,'text':obj})
              #    includes.append(word)

              if 'VBP' in tags: # 'MD' misspelled (and hence mistagged) as 'VBP'
                if (
                    word.lower().startswith('wouln') or word.lower().startswith('couln') or word.lower().startswith('shouln')
                ):     
                  parts.append({'tag':'MD','word':head,'text':obj}) # treat 'VBP' as 'MD'
                  includes.append(word)

              if 'NP' in tags: # for debugging only!
                #print('NP:', word)
                #print('head:',head)
                nouns.append({'tag':'NP','word':word,'text':obj})

              #if 'SINV' in tags: # Inverted declarative sentence (eg Had it been...)
              #  parts.append({'tag':'SINV','word':head,'text':obj}) 

              if 'VP' in tags:
                verbs.append({'tag':'VP','word':word,'text':obj})
                if 'wish' in word.lower() or 'shoul' in word.lower(): # "in"(not "==")
                  #print(word,' | ', head)
                  if head in word or len(word_tokenize(head)) < 2:
                    head = word
                  parts.append({'tag':'VP','word':head,'text':obj})
                  includes.append(word)

              if 'VBD' in tags: 
                if 'were'==word.lower() or 'was'==word.lower() or 'had'==word.lower():
                  if word not in head or len(word_tokenize(head)) < 2:
                    head = scrap_from(obj,item)
                  parts.append({'tag':'VBD','word':head,'text':obj})
                  includes.append(word)
                
              if 'VBN' in tags: # parser mistakes 'VBN' for 'VBD'(past) 
                if 'had'==word.lower():
                  if word not in head or len(word_tokenize(head)) < 2:
                    head = scrap_from(obj,item)
                  parts.append({'tag':'VBN','word':head,'text':obj})
                  includes.append(word) 

              if 'VBG' in tags:  
                if 'having'==word.lower():
                  if word not in head or len(word_tokenize(head))< 2:
                    head = scrap_from(obj,item)
                  parts.append({'tag':'VBG','word':head,'text':obj})
                  includes.append(word)


              if 'RB' in tags: 
                if 'even'==word.lower():
                  #if word not in head:
                  #  head = scrap_from(obj,item)
                  parts.append({'tag':'RB','word':head,'text':obj}) # treat 'RB' as 'MD'
                  includes.append(word)
                      
              if 'IN' in tags: 
                if (
                    'if' == word.lower() or 'unless'== word.lower() or # 'under', 'given' ??
                    'with'==word.lower() or 'without'==word.lower()
                ): 
                  #if word not in head:
                  head = scrap_from(obj,item)
                  parts.append({'tag':'IN','word':head,'text':obj})
                  includes.append(word)     

              if 'CC' in tags:
                if (
                    'but' == word.lower()
                ): 
                  phrase = scrap_from(obj,item)
                  if len(tags) > 1:   #   unless first
                    phrase = phrase[4:]# 3+1=leading 'but '
                  #print(phrase, head)  
                  if len(phrase)>0 and phrase not in head:
                    head = phrase
                  parts.append({'tag':'CC','word':head,'text':obj})
                  includes.append(word)

              tag = tags[-1]
              phrase = words[-1]
              if 'S'==tag or 'CC'==tag:
                cut = True
                for include in includes:
                  if include in phrase:
                    cut = False
                if cut and phrase not in cutouts:
                  cutouts.append(phrase)
        return

    extract(obj)
 
    #print(cutouts)
   
    anteced,conseqs = combine(parts,anteced,conseqs)
    anteced,conseqs = combine(parts,anteced,conseqs)

    return conseqs, anteced, nouns, verbs

In [ ]:
from nltk.tokenize import word_tokenize

def separate_mod(md):
  phrases = [md]  
 
  for phrase in phrases:
    if 'if ' in phrase.lower(): # 'would ask if I did something wrong"
      phrase = phrase[0:phrase.lower().index('if ')]
    if ' wish ' in phrase.lower(): # 'wish I could say I wouldn't do it"
      phrase = phrase[0:phrase.lower().index(' wish ')]
    if 'even ' in phrase.lower(): # 'even with a lot of money..","even if"
      phrase = phrase[0:phrase.lower().index('even ')]    
    #if ' with' in phrase.lower(): # 'without knowing if I can count on.."
    #  phrase = phrase[0:phrase..lower().index(' with')]
    
    if 'will have been' in phrase.lower() or 'would\'ve' in phrase.lower() or 'wouldn\'ve' in phrase.lower():
      print('phrase:',phrase)
      return phrase

    words = word_tokenize(phrase)
    for word in words:
      if (
        'would'==word.lower() or 'wouldn'==word.lower() or 'wouldn\'t'==word.lower() or 'woulnd\'t'==word.lower() or 
        'could'==word.lower() or 'could\'t'==word.lower() or 'coulnd\'t'==word.lower() or #'can'==word.lower() or
        'might'==word.lower() or 'mightn\'t'==word.lower() or 'mighn\'t'==word.lower()  or 'may'==word.lower() or
        #'should'==word.lower() or 'shouldn\'t'==word.lower() or 'shoulnd\'t'==word.lower() or 
        'd'==word.lower() or word.lower().rstrip().endswith('\'d') or 'ought'==word.lower()
      ):
        return phrase[phrase.lower().index(word.lower()):]
  return ''     

In [ ]:
def combine(parts,anteced,conseqs):
 
  for part in parts:
      tag = part['tag']
      clause= part['word']    
      tockenized = word_tokenize(clause)
      tokens = set(tockenized)

      if part['tag'] == 'MD':
        exclude = False
        for antecedent in sorted(anteced,key=len):
          if antecedent in clause: 
            exclude = True
            break
        for consequent in reversed(sorted(conseqs,key=len)):
          over = separate_from(consequent, clause)
          if over != consequent:
            exclude = True
            break
        if not exclude and clause not in conseqs: 
          conseqs.append(clause)
        #print(tag,':',clause,' [out of] ', len(parts),'-exclude?-',exclude)  
      
      if len(tokens) > 1: # eg. 'But', 'Nonetheless'
        if part['tag'] in 'SINV VBD VBN VP IN RB CC':
          exclude = False 
          for antecedent in reversed(sorted(anteced,key=len)):
            over = separate_from(antecedent,clause)
            if over != antecedent:
              if over.lower().startswith('if ') or over.lower().startswith('i wish ') : # allows "Had I have my wish"
                exclude = True
                break    
          if not exclude and clause not in anteced:
            anteced.append(clause)
          #print(tag,':',clause,' [out of] ',len(parts),'-exclude?-',exclude)  

  return anteced, conseqs

In [ ]:
from nltk import sent_tokenize

def scrap_from(obj, item=None): 
  word = str() 
  prev = str() 
  for head in obj: # head word
    if item == None or head['word'] == item['word']: 
      word = ' '# start from specified item fragment 
    if len(word) > 0: 
      word = word.strip() 
      if (not (  
              word.lower()=='\'m' or word.lower()=='\'ll' or
              word.lower()=='\'s' or word.lower()=='d' or # I'd be...
              word.lower()=='\'re' or word.lower()=='\'ve' or word.lower()=='n\'t') and  
          not (prev.lower().endswith('s') and word.lower()=='\' ') and # before\':peoples'
          not (word.lower() in '+.: m k x' and prev[-1:].isnumeric()) and # eg:'10:30am', '3M'
          not (prev.lower() in '+' and word.isnumeric()) and
          not (
              prev.lower() in '¥$€£&@({[/---#'  or  # after(eg 'S& P')
              word.lower() in '^~*:&%)}]/---...,;' # before(eg '% 5 M')
          )
      ): 
        word = word + ' ' # add space, except the aformentioned cases
      next = sent_tokenize(head['word'])[0]
      word = word + next  # add next sentence
      prev = word                 
  word = normalize(word)     
  #print(word)    
  return word

In [ ]:
def separate_children(obj):
    arr = list()

    def separate(obj, arr):
        
        if isinstance(obj, dict):

            for k, v in obj.items():
              if k != 'children':
                if isinstance(v, (dict, list)):
                    separate(v, arr)
                elif k == 'word':      
                    arr.append(v)

        elif isinstance(obj, list):
            for item in obj:
                separate(item, arr)
         
        return arr

    results = separate(obj, arr)
    return results

In [ ]:
def extract_constituents(prediction):
 for k, v in prediction.items():
  if k=='hierplane_tree':
    for ke, val in v.items():
      if ke=='root':
        for key, value in val.items():
          if key == 'children':
            return extract_modal(value)  

In [ ]:
from nltk.tokenize import word_tokenize

def simplify(constit):
  if isinstance(constit, str):
    word = normalize(constit)
    return word
   
  constit = constit['text']

  gist = list();
  unique = set() 
  for c in constit:
      p = separate_children(c) # list of c['word']s
      for head in sorted(p,key=len):
        #print(head) # head phrase (few words or one char ',')
        tokenized = word_tokenize(head)
        tokens = set(tokenized) #  without duplicates (eg ',')
        if len(unique.intersection(tokens)) <= len(tokens):
          unique.update(tokens) # without duplicate tokens!!
          gist.extend(tokenized)
  #print(gist)     
  result = str()
  prev = str()
  for word in gist:
    word = word.strip()
    if (
        not (
             word.lower()=='\'m' or word.lower()=='\'ll' or 
             word.lower()=='\'s' or word.lower()=='d' or # I'd be doing...
             word.lower()=='\'re' or word.lower()=='\'ve' or word=='n\'t'
        ) and
        not (prev.lower().endswith('s') and word.lower()=='\' ') and # before\':peoples'
        not (word.lower() in '+.: m k x' and prev[-1:].isnumeric()) and # after '10:30am'
        not (prev.lower() in '+' and word.isnumeric()) and
        not (prev.lower()=='i' and word.lower()=='d') and # ID - identification
        not  
        (
          prev.lower() in '¥$€£&@({[/---#' or  # ' ' after
          word.lower() in '^~*:&%)}]/---...,;'# or before
        )
    ): 
      result = result + ' '
      #print(prev,'-->',word)
    prev = word
    result = result + word 
  result = normalize(result)   
  #print(phrase,':',result)  
  return result

In [ ]:
import re
regex = '\\\'\s+([^\']+)\s+\\\''
def compact(fragment):
# specify the number of replacements by changing the 4th argument
  result = re.sub(regex, '\''+r'\1'+'\'' , fragment)
  return result

In [ ]:
def normalize(fragment):
  r = compact(fragment).strip()
  r = r.replace('   ',' ') 
  r = r.replace('  ',' ')
  r = r.replace(' ,',',') 
  r = r.replace(' .','.')
  r = r.replace('Mr.','Mr. ')
  r = r.replace('Ms.','Ms. ')
  r = r.replace('Mrs.','Mrs. ')
  r = r.replace('.  ','. ') # Mr./Ms./Mrs.
  r = r.replace('... ','...')
  r = r.replace(' --- ','---')
  r = r.replace(' -- ','--')
  r = r.replace(' - ','-')
  r = r.replace(' !','!')
  r = r.replace(' ?','?')
  r = r.replace('$ ','$')
  r = r.replace('# ','#')
  r = r.replace(' %','%')
  r = r.replace('( ','(')
  r = r.replace(' )',')')
  r = r.replace(' ;',';')
  r = r.replace(' :',':') 
  r = r.replace(': ',':') #50/50 - carefull!
  r = r.replace(' nt','nt')
  r = r.replace(' (k)','(k)') #401(k)
  r = r.replace(' (tm)','(tm)')
  r = r.replace('n na ','nna ')
  r = r.replace(' \'m ','\'m ')
  r = r.replace('s \' ','s\' ') # was 'neurtralized'?
  r = r.replace(' \'s ','\'s ') # a 'surgical' strike!
  r = r.replace(' \'d ','\'d ')
  r = r.replace(' \'re ','\'re ')
  r = r.replace(' \'ve ','\'ve ')
  r = r.replace(' \'ll ','\'ll ')
  r = r.replace(' n\'t','n\'t') # what about UPPER()?
  r = r.replace(' \' ',' \'')
  r = r.rstrip()
  return r

In [ ]:
def defragment(fragments, text, original, removed):
  result = []
  candid = []
  defrag = []
  prev = str() # previous fragment
  end = 0 # end of previous fragment  
  for f in sorted(fragments,key=len):
    start= 0 # start of next fragment
    r = normalize(f)
    #print(prev+' | '+r)
    try: 
      start = text.index(r)
    except ValueError:
      #print('\n{'+r+'} not in:\n '+text) # this is now in offset()
      #continue
      pass
    if start > end: # assume no overlap with prev (ie separte fragment) 
      end = start + len(r)
      prev = r
      if end <= len(text):
        candid.append(overlay(start, {'labels':0, 'text':r, 'start':start, 'end':end}, original, removed))  
    elif start <= end: # next fragment is located before prev end
      if start >= 0:
        end = start + len(r)
        prev = r
        if end <= len(text):
          candid.append(overlay(start, {'labels':0, 'text':r, 'start':start, 'end':end}, original, removed))
  
  if len(candid) == 1:
    candid[0].update(labels = 1) # gold  

  result.extend(candid)

  return result       

In [ ]:
def overlay(starting, fragment, original, removed):
  assert isinstance(fragment, dict)

  base = offset(fragment['text'], starting, original)
  text = base[0]
  start= base[1]
  end = base[2]

  for r in removed:
    #print(r[0],start,r[1],end)
    if r[0] <= start:
      start=start + 1
      #print(start,text,end)
    if r[0] <= end:
      end = end + 1
      #print(start,text,end) 
    
  fragment.update(text = text)  
  fragment.update(start=start) 
  fragment.update(end = end) 

  return fragment

In [ ]:
def offset(text, starting, original):
  #print(starting, text)
  start=0
  end = 0
  prev = str()
  for c in text:
    try: 
      start = original.index(prev + c, starting)
      end = start + len(prev) + len(c) - 1
      #print(c, start, prev, end)
    except ValueError:
      #print(c, start, prev, end)
      if c==' ': # eg.'Pfizer (NYSE: '+'PFE)'
        last = original.find(prev, starting)
        if last > -1: # back 1 space helped
          prev = prev # + c
          continue    # skip 1 space
        else:   
          if end >= 0: # reverse from end
            start = end - len(prev) + 1
            text = original[start:end+1]
            #print(start, prev, end, text)
          else:
            print('\n\"'+prev+'\" not from '+str(starting) + ' in:'+original)
          return text, start, end  
      else: 
        last = original.find(prev+' '+c, starting)
        if last > -1:                  # text=="had I had advice... 20+ years ago"
          prev = prev + ' ' + c        # prev=="had I "
          continue
        else:                          
          last = original.find(prev+c) # with space
          if last == -1:       
            last = original.find(prev) # w/o space
          if last > 0:                
            start = last-1
            end = start+len(prev)+len(c) 
            prev = prev+c
            #print(c,start,prev,end,text)
            continue
          if start >= 0: # reverse from start
            end = start + len(prev) + len(c)
            text = original[start:end]
            #print(c,start,prev,end,text)
          else:
            print('\n\"'+prev+c+'\" not from '+str(starting) + ' in:'+original)  
          return text,start,end 

    prev = prev + c

  #print(start, prev, end, text)  
  return text, start, end  

In [ ]:
def record(original):
  base=[(i, c) for i, c in enumerate(original)]
  return base

In [ ]:
def contract(text, base):
  removed = []
  open = False
  for i in range(len(base)-1):
    sequence = base[i][1]
    if i < len(base) - 1:
      sequence = sequence + base[i+1][1]
    if sequence==' \"' or sequence.startswith('\"'):
      open = True                 
    elif sequence=='\" ' or sequence.endswith('\"'):
      open = False    
    if text.find(sequence,base[i][0]-i)==-1:
      if open and sequence != ' \"':
        removed.append(base[i])
  return removed

In [ ]:
def separate_from(md, pa):
  index = -1
  if pa in md and len(md) > len(pa):
    index = md.find(pa)
    if index > 0 :
      md = md[0:index]
      #print('|'+md+'{...}')
    elif index == 0:
      md = md.replace(pa,'')
      #print('|{...}'+md)  
  return md     

In [ ]:
def separate_right(md, pa): 
  suffix = str()
  index = -1
  if pa in md and len(pa) < len(md):
    index = md.rfind(pa)    # rfind for highest index 
    #print(len(pa),index,len(md))
    if index + len(pa) < len(md):
      suffix = md[index + len(pa):]  
  return suffix

In [ ]:
def separate_left(md, pa):
  prefix = str()
  index = -1
  if pa in md and len(md) > len(pa):
    index = md.find(pa)
    if index > 0 :
      prefix = md[0:index]
    elif index == 0:
      prefix = md.replace(pa,'') # trim pa from left
  return prefix

In [ ]:
def separate(modal,past):
  for md in sorted(modal,key=len):
    for pa in reversed(sorted(past,key=len)):
      #print(md,' | ',pa)
#'For example, people diagnosed with colon cancer might typically live beyond a decade, compared to just seven months...
                # | if they had been diagnosed 40 years ago.'

      if pa in md:
        if md.find(pa)>0:
          new = md[0:md.index(pa)]
        else:
          new = md.replace(pa,'')
        new = new.strip()
      
        if len(new)==0: # md == pa
          continue
          #new = separate_mod(md) # 'might have lived beyond a decade'

        if len(new)>0:
          if new == md: # separate_mod(md): empty  
            if md in modal:
              modal.remove(md)
              print('remove [C]:',md,'[b/c: A={C}]')
            continue  

          #print(new)
          if len(word_tokenize(new)) > 1 and not (
            new.lower().lstrip().startswith('if ') or
            new.lower().lstrip().startswith('wish ')
          ):
            if md in modal:
              modal.remove(md)  
              print('replace[C]:',md,'[with:]',new)
            if new not in modal:
              modal.append(new)
              print('append [C]:',new)
            if md in past:
              past.remove(md)
              print('remove [A]:',md,'[C]:',new) 

            md = new  

      if md in pa:
        if pa.find(md)>0:
          new = pa[0:pa.index(md)]
        else:
          new = pa.replace(md,'')  
        new = new.strip()    

        if len(new)==0: # md == pa
          if pa in past:
            past.remove(pa)
            print('remove [A]:',pa,'[C]:',md)

        if len(new) > 0: 
          if new.startswith(', '):
            new = new[2:]
        
        if len(new)>0: # md is pa or part of pa
          if len(word_tokenize(new)) > 1 and not (
            new.lower().lstrip().startswith('if ') or # is pa
            new.lower().lstrip().startswith('wish ')
          ):
            if pa in past:
              past.remove(pa)
              print('remove [A]:',pa,'[C]:',md)
            if pa not in modal: # pa is not md and md is not pa
              new = separate_mod(pa) # separate md part from pa
              if len(word_tokenize(new)) > 1:
                if new not in modal:  
                  modal.append(new)
                  print('replace[C]:',md,'[with:]',new) 

                prefix = separate_left(pa,new)
                suffix = ', ' # default
                last = prefix.rfind(suffix)
                if last > 0:
                  suffix = prefix[last:]
                  prefix = prefix[0:last]
                  if len(word_tokenize(prefix)) > 1:
                    if prefix not in past:
                      past.append(prefix)
                      print('append [A]:',prefix)

            continue

          if new != pa and len(word_tokenize(new)) > 1 and (',' in new or not (
            new.lower().rstrip().endswith(' wish') or # "... you wish"
            new.lower().rstrip().endswith(' and') or 
            new.lower().rstrip().endswith(' that') or
            new.lower().rstrip().endswith(' this') or
            new.lower().rstrip().endswith(' these') or
            new.lower().rstrip().endswith(' those') or
            new.lower().rstrip().endswith(' what') or
            new.lower().rstrip().endswith(' when') or
            new.lower().rstrip().endswith(' whom') or
            new.lower().rstrip().endswith(' who') or
            new.lower().rstrip().endswith(' why')
          )):
            if (new.lower().rstrip().endswith(', that') or
                new.lower().rstrip().endswith(', this') or
                new.lower().rstrip().endswith(', these') or
                new.lower().rstrip().endswith(', those')
            ):
              mark = new.rfind(',')
              if mark > -1:
                new = new[0:mark]
              mark = new.rfind(';')
              if mark > -1:
                new = new[0:mark]           
            if pa in past: 
              past.remove(pa)
              print('remove [A]:',pa)
            if new not in past:
              print('replace[A]:',pa,'[with:]',new)
              past.append(new)          

In [ ]:
def deoverlap(past):
  for pal in sorted(past,key=len): # long/short
    for pas in reversed(sorted(past,key=len)):
      if pal == pas:
        continue

      #print(pal,'|',pas)
      prefix = separate_left(pal,pas) # pal = prefix + pas [+suffix]
      if len(prefix)>0 and prefix != pal and pal in past:
        #print('prefix:',prefix)
        if prefix.lower() in 'is, was, ; : ':  
          past.remove(pal)
          print('remove [x]:', pal)
        elif prefix.lower() in 'if wish ':  
          past.remove(pas)
          print('remove [x]:', pas)

      suffix=separate_right(pal,pas).strip()  # suffix = pal - prefix - pas 
      if len(suffix)>0 and suffix!=prefix: 
        #print('suffix:', suffix)
        if suffix.lower().strip() in ', i , it, you , we , she , he , they , that , this , these , those , which , what , when , whom , who\'s , and , why , as':
          new = pal[0:pal.rfind(suffix)] # trim suffix
          if pal in past:
            past.remove(pal)
            print('remove [x]:', pal, '[keep]:',new)             
          if new not in past:
            past.append(new) 
            print('replace[X]:',pal,'[with]:',new) 

In [ ]:
def focus(spans):
  for span1 in spans:
    text1 =span1['text']
    start1=span1['start']
    end1 = span1['end']
    if start1<0 or end1<0:
      print(span1)

    for span2 in spans:
      text2 =span2['text']
      start2=span2['start']
      end2 = span2['end']
      if start2<0 or end2<0:
        print(span2)

      if text1==text2:
        continue

      if start1 <= start2 and start2 < end1: 
        #print(start1,text1,end1,start2,text2,end2)

        end = min(end1,end2)
        if end1 > end and text1[end:end1].lower() in ', once , i , you , we , she , he , they , that , this , these , those , which , what , whom , who\'s , when, and , why ':
          if span1 in spans:
            spans.remove(span1)
        if end2 > end and text2[end:end2].lower() in ', once , i , you , we , she , he , they , that , this , these , those , which , what , whom , who\'s , when, and , why ':
          if span2 in spans:
            spans.remove(span2)

In [ ]:
def chunk(original):
  verbs = list()
  modal = list()
  nounm = list()
  past = list() 
  nounp = list()
  nouns = list()

  base = record(original)
  original = original.replace('\"','')
  removed = contract(original, base)
  text = normalize(original) # remove extra spaces

  prediction = predictor.predict(sentence=text)
  mod, pas, nou, ver = extract_constituents(prediction)
 
  for v in ver:
    vr = simplify(v) 
    if vr not in verbs:
      verbs.append(vr)

  for u in nou:
    nu = simplify(u) 
    if nu not in nouns:
      nouns.append(nu) 

  if len(mod)==0: # if no modal('MD') detected 
    for p in pas:
      pa = simplify(p)
      if pa not in past:
          past.append(pa) 

  for m in reversed(sorted(mod,key=len)):
    md = simplify(m) # get one string gist
    if len(md)==0:
      continue
    #print('md: '+md)
    
    for nm in reversed(sorted(nouns,key=len)):  
      if nm in md:   
        nounm.append(nm)
        nouns.remove(nm)

    for p in sorted(pas,key=len):
      pa = simplify(p)
      if len(pa)==0:
        continue
      #print('pa: ',pa)

      for np in reversed(sorted(nouns,key=len)):
        if np in pa:
          nounp.append(np)
          nouns.remove(np)

      if pa not in past and len(pa) > 0:
        past.append(pa)         

    if md not in modal and len(md) > 0:        
      modal.append(md)
  
  #print(past)
  #print(modal)

  deoverlap(past)

  separate(modal, past)

  #print(modal)
  #print(past)

  deoverlap(modal)
  deoverlap(past)

  #print(past)
  #print(modal)

  modals = defragment(modal, text, original, removed)
  pasts = defragment(past, text, original, removed)

  #if len(modals) > 1
  #  focus(modals)
  #if len(pasts) > 1
  #  focus(pasts)
  
  #if len(pasts)==0: # because antecedent can't be empty
  #  if len(modals) > 0:
  #    pasts.append(modals.pop(0)) # why 0? can be any!
  #    pasts[0].update(labels=1)   # gold
  #    if len(modals) == 1:
  #       modals[0].update(labels=1) # gold  
  print('______________________________________________________________________________________________________________________________________________________________________________________________________________________________')
  print(modals) 
  print(pasts)   
  print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')  
  
  
  return modals, pasts, verbs, nouns, nounm, nounp

In [ ]:
modals, pasts, verbs, nouns, nounm, nounp = chunk(
  original = 'For example, people diagnosed with colon cancer might typically live beyond a decade, compared to just seven months if they had been diagnosed 40 years ago.'
)   

In [ ]:
import pandas as pd
from tqdm import tqdm
from time import sleep

modal_df= pd.DataFrame(columns=['id','labels','text','start', 'end'])
past_df = pd.DataFrame(columns=['id','labels','text','start', 'end'])
simple_df = pd.DataFrame(columns=['id','labels','text'])

for ind in tqdm(test_df.index): 
  original = test_df['text'][ind]
  oid = test_df['id'][ind]

  print(oid)
  print(original)

  modals, pasts, verbs, nouns, nounm, nounp = chunk(original)

  i = 0
  j = 0
  for m in modals:
    m_row = pd.Series(data={
      'id': test_df['id'][ind]+'C'+ str(i),
      'labels': m['labels'],
      'text': m['text'] ,
      'start': m['start'],
      'end': m['end']
    }, name = str(i) )
    modal_df = modal_df.append(m_row)
    i = i+1
 
  for p in pasts:
    p_row = pd.Series(data={
      'id': test_df['id'][ind]+'A'+ str(j),
      'labels': p['labels'],
      'text': p['text'] ,
      'start': p['start'],
      'end': p['end']
    }, name = str(j) )
    past_df = past_df.append(p_row)   
    j = j+1   

  
  sleep(1)  

In [ ]:
past_df.to_csv(prefix+'antecedents.tsv', sep='\t', index=False, header=False)
modal_df.to_csv(prefix+'consequents.tsv', sep='\t', index=False, header=False)

In [ ]:
!mv /content/antecedents.tsv /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/
!mv /content/consequents.tsv /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/